# Kinematic Trajectory Optimization
## Ball throwing and catching

In [1]:
import time

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, BsplineTrajectory,
                         DiagramBuilder, KinematicTrajectoryOptimization,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MinimumDistanceConstraint, Parser, PositionConstraint,
                         Rgba, RigidTransform, Role, Solve, Sphere,
                         StartMeshcat, JacobianWrtVariable)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import (PublishPositionTrajectory,
                                        model_inspector)
from manipulation.scenarios import AddIiwa, AddPlanarIiwa, AddShape, AddWsg
from manipulation.utils import AddPackagePaths, FindResource


In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [9]:
import time

import numpy as np
from IPython.display import clear_output
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         JacobianWrtVariable, MathematicalProgram,
                         MeshcatVisualizer, PiecewisePolynomial, Solve,
                         StartMeshcat)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import plot_mathematical_program
from manipulation.scenarios import AddTwoLinkIiwa


# This one is specific to this notebook, but I'm putting it in the header to make it less distracting.
def Visualizer(MakeMathematicalProgram):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0) #DIFF
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    gripper_frame = plant.GetFrameByName("body", wsg)
    plant.Finalize()

    # MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    # meshcat.Delete()
    # meshcat.SetProperty("/Background", 'top_color', [0, 0, 0])
    # meshcat.SetProperty("/Background", 'bottom_color', [0, 0, 0])
    # meshcat.SetProperty("/Grid", 'visible', False)

    # X, Y = np.meshgrid(np.linspace(-5, 5, 35), np.linspace(-5, 5, 31))

    def visualize(q, v_Gdesired=[1.0, 0.0], t=None):
        if t:
            context.SetTime(t)
        plant.SetPositions(plant_context, q)
        diagram.Publish(context)

        clear_output(wait=True)
        J_G = plant.CalcJacobianTranslationalVelocity(plant_context,
                                                      JacobianWrtVariable.kQDot,
                                                      gripper_frame, [0, 0, 0],
                                                      plant.world_frame(),
                                                      plant.world_frame())
        # J_G = J_G[[0,2],:]  # Ignore Y.
        print("J_G = ")
        print(
            np.array2string(J_G,
                            formatter={'float': lambda x: "{:5.2f}".format(x)}))

        # prog = MakeMathematicalProgram(q, J_G, v_Gdesired)
        # result = Solve(prog)
        # plot_mathematical_program(meshcat, "QP", prog, X, Y, result=result)
        # # TODO: Add set_object to meshcat.Animation
        # if False: # meshcat._is_recording:
        #     with meshcat._animation.at_frame(
        #             v, meshcat._recording_frame_num) as m:
        #         plot_mathematical_program(m, prog, X, Y, result=result)

    return visualize


In [11]:
def MakeMathematicalProgram(q, J_G, v_Gdesired):
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(2, 'v')
    v_max = 3.0 

    error = J_G @ v - np.asarray(v_Gdesired)
    prog.AddCost(error.dot(error))
    prog.AddBoundingBoxConstraint(-v_max, v_max, v)

    return prog
vis = Visualizer(MakeMathematicalProgram)
vis(np.ones(7),np.array([0.5,0.5,0.5]))

J_G = 
[[-0.25  0.35  0.33  0.01 -0.06 -0.15  0.00]
 [ 0.45  0.55 -0.05 -0.49 -0.12  0.07 -0.00]
 [ 0.00 -0.45 -0.21 -0.05  0.03 -0.03  0.00]]


In [5]:
def trajopt_pv_pair_demo():
    meshcat.Delete()
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    parser = Parser(plant)
    AddPackagePaths(parser)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    X_WStart = RigidTransform([0.5, 0, 0.15])
    meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(.9, .1, .1, 1))
    meshcat.SetTransform("start", X_WStart)
    X_WGoal = RigidTransform([0, -0.6, 0.15])
    meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(.1, .9, .1, 1))
    meshcat.SetTransform("goal", X_WGoal)

    t_duration = 0.9035
    t_start = 0.5
    num_control_points = 11

    plant.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kIllustration))
    collision_visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(prefix="collision", role=Role.kProximity))
    meshcat.SetProperty("collision", "visible", False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), num_control_points)
    prog = trajopt.get_mutable_prog()
    
    trajopt.AddPathLengthCost(1.0)
    # Add cost for acceleration?

    trajopt.AddPositionBounds(plant.GetPositionLowerLimits(),
                              plant.GetPositionUpperLimits())
    trajopt.AddVelocityBounds(plant.GetVelocityLowerLimits(),
                              plant.GetVelocityUpperLimits())

    trajopt.AddDurationBounds(t_start, t_start + 2 * t_duration) # Assume t_duration each way
    trajopt.AddEqualTimeIntervalConstraints()

    # start constraint
    start_constraint = PositionConstraint(plant, plant.world_frame(),
                                          X_WStart.translation(),
                                          X_WStart.translation(), gripper_frame,
                                          [0, 0.1, 0], plant_context)
    trajopt.AddPathConstraint(start_constraint, 0)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, 0])

    # goal constraint
    goal_constraint = PositionConstraint(plant, plant.world_frame(),
                                         X_WGoal.translation(),
                                         X_WGoal.translation(), gripper_frame,
                                         [0, 0.1, 0], plant_context)
    
    trajopt.AddPathConstraint(goal_constraint, 0.5)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, np.round(num_control_points/2).astype(int)])

    # return to start
    trajopt.AddPathConstraint(start_constraint, 1)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, -1])

    
    # start and end with zero velocity
    start_V = np.zeros((7,1))
    end_V = np.zeros((7,1))
    
    # end_V[6]=-0.3
    trajopt.AddFixedPathVelocityConstraint(start_V, 0)
    trajopt.AddFixedPathVelocityConstraint(end_V, 0.5)
    trajopt.AddFixedPathVelocityConstraint(start_V, 1)

    # Solve once without the collisions and set that as the initial guess for
    # the version with collisions.
    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed, even without collisions!")
        print(result.get_solver_id().name())
    trajopt.SetPathInitialGuess(trajopt.ReconstructPath(result))

    PublishPositionTrajectory(trajopt.ReconstructPath(result), context, plant,
                              visualizer)
    collision_visualizer.Publish(
        collision_visualizer.GetMyContextFromRoot(context))

    return
    # collision constraints
    collision_constraint = MinimumDistanceConstraint(plant, 0.001,
                                                     plant_context, None, 0.01)
    evaluate_at_s = np.linspace(0, 1, 25)
    for s in evaluate_at_s:
        trajopt.AddPathConstraint(collision_constraint, s)

    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed")
        print(result.get_solver_id().name())

    PublishPositionTrajectory(trajopt.ReconstructPath(result), context, plant,
                              visualizer)
    collision_visualizer.Publish(
        collision_visualizer.GetMyContextFromRoot(context))
    
    

trajopt_pv_pair_demo()
